# Measure without a Loop

If you have a parameter that returns a whole array at once, often you want to measure it directly into a DataSet.

This shows how that works in QCoDeS

In [1]:
# setup
%matplotlib nbagg
import numpy as np
import qcodes as qc

<IPython.core.display.Javascript object>

## Instantiates all the instruments needed for the demo

For this tutorial we're going to use the regular parameters (c0, c1, c2, vsd) and ArrayGetter, which is just a way to construct a parameter that returns a whole array at once out of simple parameters, as well as AverageAndRaw, which returns a scalar *and* an array together.

In [2]:
from toymodel import AModel, MockGates, MockSource, MockMeter, AverageAndRaw, ArrayGetter

# now create this "experiment", note that all these are instruments 
model = AModel()
gates = MockGates('gates', model=model)
source = MockSource('source', model=model)
meter = MockMeter('meter', model=model)

# The station is a container for all instruments that makes it easy 
# to log meta-data
station = qc.Station(gates, source, meter)

# it's nice to have the key parameters be part of the global namespace
# that way they're objects that we can easily set, get, and slice
c0, c1, c2, vsd = gates.chan0, gates.chan1, gates.chan2, source.amplitude

### Only array output
The arguments to Measure are all the same actions you use in a Loop.
If they return only arrays, you will see exactly those arrays (with their setpoints) in the output DataSet

In [3]:
data = qc.Measure(
    qc.Task(c0.set, 0),
    ArrayGetter(meter.amplitude, c2[-10:10:0.2], 0.001),
    qc.Task(c0.set, 2),
    ArrayGetter(meter.amplitude, c2[-10:10:0.2], 0.001)
).run()

DataSet:
   mode     = DataMode.LOCAL
   location = '2016-10-20/12-14-31'
   <Type>   | <array_id>          | <array.name> | <array.shape>
   Measured | chan2_1             | chan2        | (100,)
   Measured | meter_amplitude_1_0 | amplitude    | (100,)
   Measured | chan2_3             | chan2        | (100,)
   Measured | meter_amplitude_3_0 | amplitude    | (100,)
acquired at 2016-10-20 12:14:31


### Output with scalars
Scalars still get treated as data, but it's not exactly natural:
- They still get put into arrays, just 1D 1-element arrays.
- They need a setpoint array, at least for now, so we make a fake one "single_set." There's something nice about this, actually, in that it makes it absolutely clear that these items are not really arrays.

In [4]:
data = qc.Measure(c0, c1, AverageAndRaw(meter.amplitude, c2[-10:10:0.2], 0.001)).run()

DataSet:
   mode     = DataMode.LOCAL
   location = '2016-10-20/12-14-35'
   <Type>   | <array_id>          | <array.name>  | <array.shape>
   Setpoint | single_set          | single        | (1,)
   Measured | gates_chan0         | chan0         | (1,)
   Measured | gates_chan1         | chan1         | (1,)
   Measured | chan2               | chan2         | (100,)
   Measured | meter_amplitude     | amplitude     | (100,)
   Measured | meter_avg_amplitude | avg_amplitude | (1,)
acquired at 2016-10-20 12:14:35
